In [46]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import Tensor

from sklearn.datasets import make_swiss_roll
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

import os

print(os.getcwd()) # dosya yolunu ver
%run ../02-UAE_for_SwissRoll/Model.ipynb
%run ../Dataset.ipynb

# Flow Model i çağır
%run ./Flow_Model.ipynb

plt.rcParams['font.size'] = 20
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

/arf/home/tunal/ondemand/PhD Thesis Starting/01_SON/Tik-4/Tez/02-SwissRoll/05-UAE_Latent_FlowMatching-Copy1


In [47]:
# Custom Transform
class NoiseTransform:
    """Add some noise."""

    def __init__(self, split_ratio=0.001, dim=3):

        self.normal_dist = split_ratio*np.random.randn(dim,)
        
    def __call__(self, x):
      return x + self.normal_dist

In [48]:
# Hyper-Parameters & Settings
dataset_size = 10000
batch_size = 500

In [49]:
# Dataset
train_dataset = SwissRollDataset(mode='train', n_samples=dataset_size)
val_dataset = SwissRollDataset(mode='val', n_samples=dataset_size)
test_dataset = SwissRollDataset(mode='test', n_samples=dataset_size)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

In [50]:
# UAE modelini çalıştır

path = '../02-UAE_for_SwissRoll/results/UAE_SwissRoll'
model = torch.load(path + '.model', weights_only=False)
model.eval()

To_Uniform(
  (encoder): Sequential(
    (0): Linear(in_features=3, out_features=400, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=400, out_features=400, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=400, out_features=400, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=400, out_features=400, bias=True)
    (10): ReLU()
    (11): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=400, out_features=400, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Linear(in_features=400, out_features=2, bias=True)
    (16): Sigmoid()
  )
  (decoder): Sequential(
    (0): L

In [51]:
# "results" klasörünü oluştur (zaten varsa hata vermez)
os.makedirs("results", exist_ok=True)

flow_name = './results/UAE_Latent_FlowMatching'
flow = Flow()

optimizer = torch.optim.Adam(flow.parameters(), 0.001)
loss_fn = nn.MSELoss()

In [52]:
train_cost = []
val_cost = []
best_val_loss = float('inf')  # Başlangıçta çok büyük bir değer

for epoch in range(5000):
    epoch_losses = []

    flow.train()
    for x, _ in train_loader:

        x_1 = model.encoder(x)  # encode edilmiş temsil (batch_size x latent_dim)
        
        x_0 = torch.rand_like(x_1)
        t = torch.rand(len(x_1), 1)

        x_t = (1 - t) * x_0 + t * x_1
        dx_t = x_1 - x_0

        optimizer.zero_grad()
        pred = flow(t=t, x_t=x_t)
        loss = loss_fn(pred, dx_t)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    mean_train_loss = sum(epoch_losses) / len(epoch_losses)
    train_cost.append(mean_train_loss)

    # Validation
    flow.eval()
    val_losses = []
    with torch.no_grad():
        for x_val, _ in val_loader:
            x_1_val = model.encoder(x_val) # encode edilmiş temsil (batch_size x latent_dim)
            
            x_0_val = torch.rand_like(x_1_val)
            t_val = torch.rand(len(x_1_val), 1)

            x_t_val = (1 - t_val) * x_0_val + t_val * x_1_val
            dx_t_val = x_1_val - x_0_val

            pred_val = flow(t=t_val, x_t=x_t_val)
            val_loss = loss_fn(pred_val, dx_t_val)
            val_losses.append(val_loss.item())

    mean_val_loss = sum(val_losses) / len(val_losses)
    val_cost.append(mean_val_loss)

    # En iyi modeli val loss'a göre kaydet
    if mean_val_loss < best_val_loss:
        print('Saved!')
        best_val_loss = mean_val_loss
        torch.save(flow, flow_name + '.model')

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1}/10000 - Train Loss: {mean_train_loss:.4f} - Val Loss: {mean_val_loss:.4f}")

# Son epoch modelini ayrı istersen:
torch.save(flow, flow_name + '_final.model')

Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Saved!
Epoch 1000/10000 - Train Loss: 0.1277 - Val Loss: 0.1268
Epoch 2000/10000 - Train Loss: 0.1279 - Val Loss: 0.1225
Saved!
Epoch 3000/10000 - Train Loss: 0.1251 - Val Loss: 0.1282
Epoch 4000/10000 - Train Loss: 0.1269 - Val Loss: 0.1259
Epoch 5000/10000 - Train Loss: 0.1268 - Val Loss: 0.1233


In [53]:
# CSV dosyasına kaydet
train_losses = train_cost  # liste veya numpy array
val_losses = val_cost

np.savetxt("results/losses.csv", 
           np.column_stack((train_losses, val_losses)), 
           delimiter=",", 
           header="train_loss,val_loss", 
           comments="")